In [1]:
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
import pandas as pd
import numpy as np

In [6]:
def data_preprocess(dta):
    dta['Date'] = pd.to_datetime(dta['Date'], format='%m/%d/%Y')
    dta = dta.set_index(dta['Date'])
    # NHLI not traded
    dta.drop(['Date', 'NHLI'], axis=1, inplace=True)
    dta.dropna(how='all', inplace=True)
    for tick in dta.columns:
        tick_series = dta[tick]
        start_pos = tick_series.first_valid_index()
        valid_series = tick_series.loc[start_pos:]
        if valid_series.isna().sum() > 0:
            dta.drop(tick, axis=1, inplace=True)

    for tick in dta.columns:
        dta[tick] = dta[tick].mask(dta[tick] == 0).ffill(downcast='infer')

    return dta[dta.index >= dta['SPY'].first_valid_index()]


In [7]:
dta = pd.read_csv('broader_stock.csv')
dta = data_preprocess(dta)


In [8]:
tick = 'ASB'

test = dta[tick]

if tick in dta.columns:
    temp = pd.concat([dta.drop([tick], axis=1), test], axis=1).dropna(axis=1)
    temp['%s_LAG' % tick] = temp[tick].shift(-5)
    temp.dropna(inplace=True)
else:
    temp = pd.concat([dta, test], axis=1).dropna(axis=1)
    temp['%s_LAG' % tick] = temp[tick].shift(-5)
    temp.dropna(inplace=True)

y = temp['%s_LAG' % tick]
cointegrat = {}
correlat = {}



In [9]:
y = pct_change(y)
cointegrat = {}
correlat = {}

for i in temp.columns[:-2]:
    x = temp[i]
    x = pct_change(x)
    score, pval, _ = coint(x, y, trend='ct')
    corr = x.corr(y)

    cointegrat[i] = pval
    correlat[i] = corr

best_coint = sorted(cointegrat, key=cointegrat.get)[:50]
best_corr = sorted(correlat, key=correlat.get, reverse=True)[:50]

intersect = list(set(best_coint) & set(best_corr))

In [10]:
union = list(set(best_coint) | set(best_corr))
intersect.append('SPY')
union.append('SPY')

In [11]:
X = temp[union]
X = X.apply(pct_change, axis=0)
Y = temp['ASB_LAG']
Y = pct_change(Y)

cutoff = int(X.shape[0] * 0.8)

X_tv, X_test = X.iloc[:cutoff], X.iloc[cutoff:]
Y_tv, Y_test = Y.iloc[:cutoff], Y.iloc[cutoff:]

## Baseline Model
$$
R_{i,t} = \sum_k \beta_k R_{k, t-l}
$$

In [12]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [13]:
alpha = np.linspace(0.001, 1000, 300)

l1reg = LassoCV(alphas=alpha, fit_intercept=True, cv=10, n_jobs=-1).fit(X_tv, Y_tv)
l1_pred = l1reg.predict(X_test)

l2reg = RidgeCV(alphas=alpha, fit_intercept=True, cv=10).fit(X_tv, Y_tv)
l2_pred = l2reg.predict(X_test)

ols = sm.OLS(Y_tv, sm.add_constant(X_tv)).fit()
ols_pred = ols.predict(sm.add_constant(X_test))

In [14]:
l1_r2 = r2_score(Y_test.values, l1_pred)
l2_r2 = r2_score(Y_test.values, l2_pred)
ols_r2 = r2_score(Y_test.values, ols_pred.values)

In [15]:
max(l1_r2, l2_r2, ols_r2)

-0.00020526102920137568

## Factor Model
$$
R_{i,t} = \sum_k \beta_k F_{k, t-l}
$$

In [16]:
from sklearn.decomposition import FactorAnalysis

In [17]:
transformer = FactorAnalysis(n_components=20, max_iter=5000, svd_method='lapack')
X_transformed = transformer.fit_transform(X_tv)

In [18]:
l1reg = LassoCV(alphas=alpha, fit_intercept=True, cv=10, n_jobs=-1).fit(X_transformed, Y_tv)
l1_pred = l1reg.predict(transformer.transform(X_test))

l2reg = RidgeCV(alphas=alpha, fit_intercept=True, cv=10).fit(X_transformed, Y_tv)
l2_pred = l2reg.predict(transformer.transform(X_test))

ols = sm.OLS(Y_tv, sm.add_constant(X_transformed)).fit()
ols_pred = ols.predict(sm.add_constant(transformer.transform(X_test)))

In [19]:
l1_r2 = r2_score(Y_test.values, l1_pred)
l2_r2 = r2_score(Y_test.values, l2_pred)
ols_r2 = r2_score(Y_test.values, ols_pred)

In [20]:
max(l1_r2, l2_r2, ols_r2)

-0.00020526102920137568

## PCA Model
$$
R_{i,t} = \sum_k \beta_k PCA_{k, t-l}
$$

In [21]:
from sklearn.decomposition import PCA

In [22]:
transformer = PCA(n_components=20)
X_transformed = transformer.fit_transform(X_tv)

In [23]:
l1reg = LassoCV(alphas=alpha, fit_intercept=True, cv=10, n_jobs=-1).fit(X_transformed, Y_tv)
l1_pred = l1reg.predict(transformer.transform(X_test))

l2reg = RidgeCV(alphas=alpha, fit_intercept=True, cv=10).fit(X_transformed, Y_tv)
l2_pred = l2reg.predict(transformer.transform(X_test))

ols = sm.OLS(Y_tv, sm.add_constant(X_transformed)).fit()
ols_pred = ols.predict(sm.add_constant(transformer.transform(X_test)))

In [24]:
l1_r2 = r2_score(Y_test.values, l1_pred)
l2_r2 = r2_score(Y_test.values, l2_pred)
ols_r2 = r2_score(Y_test.values, ols_pred)

In [25]:
max(l1_r2, l2_r2, ols_r2)

-0.00020526102920137568

## LSTM baby

In [26]:
# ML imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
import keras.backend as K
from keras.callbacks import EarlyStopping
from tensorflow.keras import initializers

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [27]:
def coeff_deter(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred) * 1e6)
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)) * 1e6)
    return 1 - SS_res / (SS_tot + K.epsilon())

In [28]:
block_size = 30

X_ml, Y_ml = [], []
for i in range(X_tv.shape[0] - block_size):
    x = X_tv.iloc[i:i+block_size].values.T.flatten()
    y = Y_tv.iloc[i+block_size-5:i+block_size].values.T
    X_ml.append(x)
    Y_ml.append(y)


X_mltest, Y_mltest = [], []
for i in range(X_test.shape[0] - block_size):
    x = X_test.iloc[i:i+block_size].values.T.flatten()
    y = Y_test.iloc[i+block_size-5:i+block_size].values.T
    X_mltest.append(x)
    Y_mltest.append(y)

In [29]:
X_ml = np.array(X_ml)
X_ml = X_ml.reshape(X_ml.shape[0], 1, X_ml.shape[1])
Y_ml = np.array(Y_ml)

X_mltest = np.array(X_mltest)
X_mltest = X_mltest.reshape(X_mltest.shape[0], 1, X_mltest.shape[1])
Y_mltest = np.array(Y_mltest)

In [30]:
initializer = initializers.glorot_normal(seed=42)
model = Sequential()
model.add(LSTM(100, input_shape=(X_ml.shape[1], X_ml.shape[2]), kernel_initializer=initializer, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(50, input_shape=(X_ml.shape[1], X_ml.shape[2]), kernel_initializer=initializer))
model.add(Dropout(0.4))
model.add(Dense(25, kernel_initializer=initializer))
model.add(Dropout(0.4))
model.add(Dense(5, kernel_initializer=initializer))
model.compile(loss='mae', optimizer='adam', metrics=[coeff_deter])

es = EarlyStopping(monitor='val_coeff_deter', mode='max', patience=5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [31]:
model.fit(X_ml, Y_ml,
            batch_size=32,
            validation_data=(X_mltest, Y_mltest),
            epochs=50,
            callbacks=[es],
            verbose=0)


In [32]:
y_pred = model.predict(X_mltest)
r2_score(Y_mltest, y_pred)

-0.07252833463933368

## LSTM w/ Factor

In [95]:
transformer = FactorAnalysis(n_components=20, max_iter=5000, svd_method='lapack')
X_transformed = transformer.fit_transform(X_tv)
X_test_tran = transformer.transform(X_test)

In [96]:
block_size = 30

X_ml, Y_ml = [], []
for i in range(X_transformed.shape[0] - block_size):
    x = X_transformed[i:i+block_size,:].T.flatten()
    y = Y_tv.iloc[i+block_size-5:i+block_size].values.T
    X_ml.append(x)
    Y_ml.append(y)


X_mltest, Y_mltest = [], []
for i in range(X_test_tran.shape[0] - block_size):
    x = X_test_tran[i:i+block_size,:].T.flatten()
    y = Y_test.iloc[i+block_size-5:i+block_size].values.T
    X_mltest.append(x)
    Y_mltest.append(y)

In [97]:
X_ml = np.array(X_ml)
X_ml = X_ml.reshape(X_ml.shape[0], 1, X_ml.shape[1])
Y_ml = np.array(Y_ml)

X_mltest = np.array(X_mltest)
X_mltest = X_mltest.reshape(X_mltest.shape[0], 1, X_mltest.shape[1])
Y_mltest = np.array(Y_mltest)

In [101]:
X_ml.shape

(4146, 1, 600)

In [102]:
Y_ml.shape

(4146, 5)

In [109]:
xtr, xva, ytr, yva = train_test_split(X_ml, Y_ml, test_size=0.2)

In [110]:
xtr.shape

(3316, 1, 600)

In [111]:
xva.shape

(830, 1, 600)

In [98]:
initializer = initializers.glorot_normal(seed=42)
model = Sequential()
model.add(LSTM(100, input_shape=(X_ml.shape[1], X_ml.shape[2]), kernel_initializer=initializer, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(50, input_shape=(X_ml.shape[1], X_ml.shape[2]), kernel_initializer=initializer))
model.add(Dropout(0.4))
model.add(Dense(25, kernel_initializer=initializer))
model.add(Dropout(0.4))
model.add(Dense(5, kernel_initializer=initializer))
model.compile(loss='mae', optimizer='adam', metrics=[coeff_deter])

es = EarlyStopping(monitor='val_coeff_deter', mode='max', patience=5)

In [99]:
model.fit(X_ml, Y_ml,
            batch_size=32,
            validation_data=(X_mltest, Y_mltest),
            epochs=50,
            callbacks=[es],
            verbose=0)


In [100]:
y_pred = model.predict(X_mltest)
r2_score(Y_mltest, y_pred)

-0.011376479877204382

## LSTM w/ PCA

In [39]:
transformer = PCA(n_components=20)
X_transformed = transformer.fit_transform(X_tv)
X_test_tran = transformer.transform(X_test)

In [40]:
block_size = 30

X_ml, Y_ml = [], []
for i in range(X_transformed.shape[0] - block_size):
    x = X_transformed[i:i+block_size,:].T.flatten()
    y = Y_tv.iloc[i+block_size-5:i+block_size].values.T
    X_ml.append(x)
    Y_ml.append(y)


X_mltest, Y_mltest = [], []
for i in range(X_test_tran.shape[0] - block_size):
    x = X_test_tran[i:i+block_size,:].T.flatten()
    y = Y_test.iloc[i+block_size-5:i+block_size].values.T
    X_mltest.append(x)
    Y_mltest.append(y)

In [41]:
X_ml = np.array(X_ml)
X_ml = X_ml.reshape(X_ml.shape[0], 1, X_ml.shape[1])
Y_ml = np.array(Y_ml)

X_mltest = np.array(X_mltest)
X_mltest = X_mltest.reshape(X_mltest.shape[0], 1, X_mltest.shape[1])
Y_mltest = np.array(Y_mltest)

In [42]:
initializer = initializers.glorot_normal(seed=42)
model = Sequential()
model.add(LSTM(100, input_shape=(X_ml.shape[1], X_ml.shape[2]), kernel_initializer=initializer, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(50, input_shape=(X_ml.shape[1], X_ml.shape[2]), kernel_initializer=initializer))
model.add(Dropout(0.4))
model.add(Dense(25, kernel_initializer=initializer))
model.add(Dropout(0.4))
model.add(Dense(5, kernel_initializer=initializer))
model.compile(loss='mae', optimizer='adam', metrics=[coeff_deter])

es = EarlyStopping(monitor='val_coeff_deter', mode='max', patience=5)

In [43]:
model.fit(X_ml, Y_ml,
            batch_size=32,
            validation_data=(X_mltest, Y_mltest),
            epochs=50,
            callbacks=[es],
            verbose=0)


In [44]:
y_pred = model.predict(X_mltest)
r2_score(Y_mltest, y_pred)

-0.03492347771075051

In [125]:
test = np.apply_along_axis(lambda x: np.cumprod(1+x)[-1], 1, y_pred)
len(test)

1015

In [64]:
y_pred

array([[ 1.5423977e-04,  2.9314016e-03, -4.2806850e-03,  3.0069626e-03,
         6.4895186e-04],
       [ 3.7038170e-03, -1.1275200e-03,  5.5413356e-04, -9.3834952e-04,
         4.6284887e-04],
       [-9.9192571e-04,  1.3953125e-03, -4.9736386e-04,  1.7951954e-03,
        -9.5262541e-05],
       ...,
       [ 3.1616448e-03, -2.1325932e-03, -1.7393162e-03,  1.5560952e-03,
         5.1253976e-04],
       [-9.1161032e-04,  8.7106199e-04, -1.5744175e-03,  8.2369539e-04,
         2.4380133e-06],
       [ 2.4724723e-04, -1.9982834e-03, -4.2590452e-04,  3.7878253e-03,
        -1.8244577e-03]], dtype=float32)

In [115]:
from statsmodels.tsa.stattools import adfuller

In [122]:
adval = adfuller(Y.iloc[-1015:])

In [123]:
adval[1]

1.3615278532780622e-20

In [124]:
1e-3

0.001

In [90]:
def measure_profit(price, indicator):
    assert len(price) == len(indicator)
    inventory = 0
    asset = 0
    record = [asset]
    for i in range(len(price)):
        trend_good = indicator[i] > 1
        p = price[i]
        if trend_good and inventory == 0:
            # buy
            asset -= p
            inventory += 1
        elif not trend_good and inventory == 1:
            # sell
            asset += p
            inventory -= 1
        elif i == len(price) - 1 and inventory == 1:
            asset += p
            inventory -= 1
        else:
            asset = record[-1]
        record.append(asset)
    return asset, record[1:]

In [91]:
ass, rec = measure_profit(temp['ASB'].iloc[-1015:].values, test)

In [92]:
ass

-7.980000000000032